# 回帰分析による因果推論
テレビCMの広告効果の推定を例に、回帰分析による因果推論を実装します。


In [1]:
import random
import numpy as np
np.random.seed(1234)
random.seed(1234)

In [2]:
from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

In [3]:
def make_data(num_data=200):
    x_1=randint(15,76,num_data) #age
    x_2=randint(0,2,num_data) #sex, 0:female,1:male
    
    e_z = randn(num_data)
    z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z
    z_prob = expit(0.1*z_base)
    Z = np.array([])
    for i in range(num_data):
        Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
        Z = np.append(Z, Z_i)
    e_y = randn(num_data)
    Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y
    df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })
    return df

In [4]:
df = make_data(num_data=200)
df.head()

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


$
\mathrm{Prob}(Z)=\mathrm{sigmoid}(0.1(x_1+10(1-x_2)-40+5\epsilon_z))
$

$
y=-x_1+30x_2+10z+80+10\epsilon_y
$

データをまとめた表を作成し、平均値を比べる

In [5]:
print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())


年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


回帰分析を実施

In [6]:
X = df[["年齢", "性別", "CMを見た"]]
Y = df["購入量"]
reg = LinearRegression().fit(X, Y)
print("係数：", reg.coef_)


係数： [-0.95817951 32.70149412 10.41327647]


- 逆確率重み付け法（IPTW）による因果推論

In [7]:
X = df[["年齢", "性別"]]
Z = df["CMを見た"]
reg = LogisticRegression().fit(X,Z)
print("係数beta：", reg.coef_)
print("係数alpha：", reg.intercept_)

係数beta： [[ 0.10562765 -1.38263933]]
係数alpha： [-3.37146523]


傾向スコア

In [8]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる
print("----")
print(Z[0:5])  # 5人ほどの正解

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
----
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: CMを見た, dtype: float64


平均処置効果(ATE)

In [9]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:, 0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)

推定したATE 8.847476810855458


- Doubly Robust法（DR法）による因果推論

In [10]:
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
Y = df["購入量"]

# 回帰の実施
reg2 = LinearRegression().fit(X, Y)

# Z=0の場合
X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 = reg2.predict(X_0)

# Z=1の場合
X_1 = X.copy()
X_1["CMを見た"] = 1
Y_1 = reg2.predict(X_1)

In [11]:
# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

# 傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


In [12]:
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()
print("推定したATE", ATE)

推定したATE 9.752775054248461
